# Analyse des données

Ce notebook est dédié à l'analyse du dataset généré depuis `data-processing.ipynb`. Nos objectifs sont :
1. L'identification des lignes et des tronçons les plus problématiques sur le réseau IDF Mobilités
2. L'identification des perturbations les plus fréquentes
3. La mise en contexte socio-géographique de ces observations
4. L'identification de potentiels problèmes dans la restitution des données par l'API IDF Mobilités

## Imports

In [1]:
# On utilise cudf.pandas pour accélérer les opérations Pandas sur GPU, optionnel

%pip install \
  --extra-index-url=https://pypi.nvidia.com \
  cudf-cu12==24.12.* \
  dask-cudf-cu12==24.12.* \
  cuml-cu12==24.12.* \
  cugraph-cu12==24.12.*

%load_ext cudf.pandas

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from dotenv import load_dotenv

load_dotenv()

True

## Statistiques descriptives

Nous commençons par produire des statistiques descriptives pour orienter la suite de notre travail. A ce stade, nous cherchons à savoir s'il sera nécessaire d'aller plus loin sur le pré-traitement des données et à identifier des axes de travail intéressants en dehors de ceux que nous avons déjà identifiés.

In [29]:
df_disruptions = pd.read_feather("../data/disruptions.feather")
df_objects = pd.read_feather("../data/objects.feather")

/opt/conda/lib/python3.12/site-packages/cudf/io/feather.py:15: UserWarning:

Using CPU via PyArrow to read feather dataset, this may be GPU accelerated in the future



### Perturbations

In [30]:
df_disruptions.head()

,disruption_id,begin,end,lastUpdate,cause,severity,title,message,file_lastUpdatedDate
0,e3fc69f8-9b31-11ee-8c7d-0a58a9feac02,20231215T110000,20241231T235900,20240402T182723,TRAVAUX,BLOQUANTE,CCEJR SUPPRIME LIGNE 68.09,<p>En raison des travaux et de l'impossibilité...,2024-12-06T13:50:22.638Z
1,24d4b560-32dc-11ef-9244-0a58a9feac02,20240625T121500,20250212T141500,20240625T124802,TRAVAUX,BLOQUANTE,Métro 14 : Travaux - Arrêt Villejuif - Gustave...,"<p>En raison de travaux, l'arrêt Villejuif - G...",2024-12-06T13:50:22.638Z
2,863dcb28-5f8e-11ef-9a4d-0a58a9feac02,20240708T070000,20241208T230000,20241205T120845,TRAVAUX,BLOQUANTE,Fermeture Chemin de la Noue Rousseau -Plessis-...,<p>En raison de travaux et de la fermeture d’u...,2024-12-06T13:50:22.638Z
3,a428d2aa-65ff-11ef-ac31-0a58a9feac02,20240909T000000,20241213T235900,20240829T141016,TRAVAUX,BLOQUANTE,🚧 5192 5196 - Travaux : Collège Champollion no...,<p><strong>🚧 Perturbations #Ligne5192 #Ligne51...,2024-12-06T13:50:22.638Z
4,13188de8-6aa3-11ef-abce-0a58a9feac02,20240904T120000,20241231T230000,20240904T115010,PERTURBATION,BLOQUANTE,Travaux à Mantes-La-Jolie,"<p>En raison de travaux à Mantes-la-Jolie,<br>...",2024-12-06T13:50:22.638Z


On remarque immédiatement (c'est quelque chose que nous avions déjà identifié lors d'un travail préliminaire de compréhension de l'API IDF Mobilités) que tous les travaux ne sont pas indiqués comme tels dans la colonne `cause`. Un travail supplémentaire est envisageable pour re-catégoriser les types de perturbations.

In [40]:
print("Début de la collecte :", df_disruptions["file_lastUpdatedDate"].min())
print("Nombre de perturbations :", len(df_disruptions))
print("Nombre de perturbations (id uniques) :", len(df_disruptions.drop_duplicates(subset=["disruption_id"])))
print("Causes :", df_disruptions["cause"].unique())
print("Séverités :" ,df_disruptions["severity"].unique())

Début de la collecte : 2024-12-06 13:50:22.638000+00:00
Nombre de perturbations : 30177
Nombre de perturbations (id uniques) : 19864
Causes : ['TRAVAUX' 'PERTURBATION' 'INFORMATION']
Séverités : ['BLOQUANTE' 'PERTURBEE' 'INFORMATION']


Une perturbation récurrente (par exemple : des travaux répartis sur plusieurs weekends) aura le même id mais occupera plusieurs lignes dans notre jeu de données.

Il est très clair que le gros des informations non temporelles sur une perturbation donnée est compris dans les colonnes `title` et `message`. Nous allons devoir faire du **traitement du langage naturel** pour augmenter ces données.

Avant d'aller plus loin, convertissons les colonnes de dates.

In [32]:
df_disruptions['begin'] = pd.to_datetime(df_disruptions['begin'], format='%Y%m%dT%H%M%S')
df_disruptions['end'] = pd.to_datetime(df_disruptions['end'], format='%Y%m%dT%H%M%S')
df_disruptions['lastUpdate'] = pd.to_datetime(df_disruptions['lastUpdate'], format='%Y%m%dT%H%M%S')
df_disruptions['file_lastUpdatedDate'] = pd.to_datetime(df_disruptions['file_lastUpdatedDate'])

In [64]:
df_dec2024 = df_disruptions[
    (df_disruptions['begin'].dt.year == 2024) &
    (df_disruptions['begin'].dt.month == 12)
].copy()

df_dec2024['day'] = df_dec2024['begin'].dt.day
df_dec2024_daily_cause = (
    df_dec2024
    .groupby(['day', 'cause'], as_index=False)
    .size()
    .rename(columns={'size': 'count'})
)

fig_time_dec2024_cause = px.line(
    df_dec2024_daily_cause,
    x='day',
    y='count',
    color='cause',
    markers=True,
    title='Nombre de pertubations en Décembre 2024 par jour et par cause'
)
fig_time_dec2024_cause.update_layout(
    xaxis_title='Jour de Décembre 2024',
    yaxis_title='Nombre de perturbations',
    hovermode='x'
)
fig_time_dec2024_cause.show()

Pour rappel, notre collecte de données a débuté le 6 décembre en milieu de journée. On remarque un pic de perturbations à Noël et une hausse des perturbations déjà annoncée pour le 31.

In [58]:
df_disruptions['duration_hours'] = (
    df_disruptions['end'] - df_disruptions['begin']
).dt.total_seconds() / 3600

In [59]:
def describe(df, column, title):
    mean = df[column].mean()
    median = df[column].median()
    std = df[column].std()
    quantiles = df[column].quantile([0.0, 0.05, 0.25, 0.5, 0.75, 0.95, 1.0])

    print(title)
    print(f"  Mean:     {mean:.2f}")
    print(f"  Median:   {median:.2f}")
    print(f"  Std:      {std:.2f}")
    print("  Quantiles (0%, 25%, 50%, 75%, 100%):")
    print(quantiles)

In [60]:
describe(df_disruptions, "duration_hours", "Durée des perturbations en heures")

Durée des perturbations en heures
  Mean:     254.14
  Median:   4.00
  Std:      1655.65
  Quantiles (0%, 25%, 50%, 75%, 100%):
0.00         0.000278
0.05         0.016667
0.25         0.414722
0.50         4.000000
0.75         9.666667
0.95      1250.571944
1.00    122735.983333
Name: duration_hours, dtype: float64


La distribution est extrêmement asymétrique, certaines perturbations durent des jours et des jours. Essayons de visualiser la répartitions de celles de moins de 24 heures.

In [63]:
lower_bound = 0
upper_bound = 24

df_duration_filtered = df_disruptions[
    (df_disruptions['duration_hours'] >= lower_bound) &
    (df_disruptions['duration_hours'] <= upper_bound)
]

# Boxplot of durations (hours) without extreme values
fig_duration_box = px.box(
    df_duration_filtered, 
    y='duration_hours',
    title='Répartition des durées des perturbations (heures)'
)
fig_duration_box.update_layout(yaxis_title='Durée (heures)')
fig_duration_box.show()

Le fait de ne garder que les perturbations d'une durée inférieure à 24h a réduit la leur durée médiane de une heure.

In [61]:
df_disruptions['title_length'] = df_disruptions['title'].astype(str).apply(len)
df_disruptions['message_length'] = df_disruptions['message'].astype(str).apply(len)

describe(df_disruptions, "title_length", "Longueur des titres des perturbations")
describe(df_disruptions, "message_length", "Longueur des messages d'info-trafic des perturbations")

Longueur des titres des perturbations
  Mean:     49.59
  Median:   48.00
  Std:      11.51
  Quantiles (0%, 25%, 50%, 75%, 100%):
0.00      1.0
0.05     35.0
0.25     44.0
0.50     48.0
0.75     56.0
0.95     66.0
1.00    598.0
Name: title_length, dtype: float64
Longueur des messages d'info-trafic des perturbations
  Mean:     472.31
  Median:   320.00
  Std:      388.65
  Quantiles (0%, 25%, 50%, 75%, 100%):
0.00      48.0
0.05     209.0
0.25     276.0
0.50     320.0
0.75     406.0
0.95    1551.0
1.00    2241.0
Name: message_length, dtype: float64


Ces chiffres sont encourageants, nous aurons à priori assez de données en langage naturel à traiter pour augmenter ce jeu de données.

In [62]:
avg_duration_by_cause = (
    df_disruptions.groupby('cause')['duration_hours']
    .mean()
    .reset_index(name='avg_duration_hours')
)

fig_avg_duration_by_cause = px.bar(
    avg_duration_by_cause, 
    x='cause', 
    y='avg_duration_hours', 
    title='Durée moyenne en fonction de la cause'
)
fig_avg_duration_by_cause.show()

# Average duration by severity
avg_duration_by_severity = (
    df_disruptions.groupby('severity')['duration_hours']
    .mean()
    .reset_index(name='avg_duration_hours')
)

fig_avg_duration_by_severity = px.bar(
    avg_duration_by_severity, 
    x='severity', 
    y='avg_duration_hours', 
    title='Durée moyenne en fonction de la sévérité'
)
fig_avg_duration_by_severity.show()

In [55]:
df_disruptions[df_disruptions["cause"] == "INFORMATION"]["message"].head(10).apply(print)

<p>Bonjour,</p><p>&nbsp;</p><p>Suite aux travaux prolongés sur Vaux-le-Pénil, du 21 nov. 2024 au 6 déc. 2024.<br><br>L'arrêt suivant ne sera pas desservi :&nbsp;</p><p>&nbsp;</p><ul><li><span style="color:hsl(0,75%,60%);"><strong>Vaux-le-Pénil - Saint Just Clémenceau </strong>(dans les deux sens)<strong> </strong><i><strong><u>de 8h00 à 17h00 uniquement.</u></strong></i></span></li></ul><p>&nbsp;</p><p>Nous vous prions de bien vouloir nous excuser pour la gêne occasionnée.&nbsp;</p>
<p><span>⚠️🚍</span><span style="color:rgb(29,155,240);"><span>#InfoTrafic</span></span><span> - Lignes E, M et 12 -À partir du mercredi 04 Septembre 2024 : Tous les mercredis de 5h00 à 16h00</span><br>&nbsp;</p><p><span>Changement de Terminus Arrivée/ Départ :</span></p><p><span><strong>❌Arrêt Gare de Chatou parvis Nord non desservi</strong></span></p><p><span><strong>✅Report arrêt Gare de Chatou parvis Sud</strong></span></p><p><br>&nbsp;</p><p><span>Veuillez nous excuser de la gêne occasionnée.</span></p>

163    <NA>
867    <NA>
872    <NA>
874    <NA>
880    <NA>
881    <NA>
899    <NA>
900    <NA>
901    <NA>
906    <NA>
Name: message, dtype: object

Les informations semblent généralement correspondre à des déviations pour travaux, mais nous devrons augmenter le jeu de données pour en être certains. Essayons de n'observer que les perturbations qui ne sont ni des travaux, ni des informations.

In [70]:
uncategorized_disruptions = df_disruptions[df_disruptions["cause"] == "PERTURBATION"]
describe(uncategorized_disruptions, "duration_hours", "Durée des perturbations en heures")

Durée des perturbations en heures
  Mean:     68.94
  Median:   2.50
  Std:      677.49
  Quantiles (0%, 25%, 50%, 75%, 100%):
0.00        0.000278
0.05        0.016667
0.25        0.016667
0.50        2.500000
0.75        7.000000
0.95       21.421667
1.00    26946.983333
Name: duration_hours, dtype: float64


In [78]:
inf_2mn = uncategorized_disruptions[uncategorized_disruptions['duration_hours'] * 60 <= 2]
taux_inf_2mn = (len(inf_2mn) / len(uncategorized_disruptions)) * 100
print(f"{taux_inf_2mn:.2f}% des perturbations de notre jeu de données (hors travaux et informations) durent deux minutes ou moins")

26.11% des perturbations de notre jeu de données (hors travaux et informations) durent deux minutes ou moins


In [79]:
inf_2mn = df_disruptions[df_disruptions['duration_hours'] * 60 <= 2]
taux_inf_2mn = (len(inf_2mn) / len(df_disruptions)) * 100
print(f"{taux_inf_2mn:.2f}% des perturbations de notre jeu de données (tout compris) durent deux minutes ou moins")

19.98% des perturbations de notre jeu de données (tout compris) durent deux minutes ou moins


Compte tenu de cette information, nous pouvons nous inquiéter quant à l'exhaustivité de notre jeu de données, celui-ci étant constitué sur la base d'appels à l'API IDF Mobilités toutes les deux minutes (et même moins fréquents en pratique). Nous irons malgré tout au bout de notre analyse, en ayant conscience que notre travail est potentiellement biaisé du fait d'un trop faible échantillonnage.

### Objets du réseau (lignes, arrêts, tronçons...)

L'API IDF Mobilités renvoie la liste des objets du réseaux affectés par une ou plusieurs perturbations. Tous ces "objets" sont identifiés de manière unique et listés ici.

In [80]:
df_objects.head()

,line_id,line_name,line_shortName,line_mode,line_networkId,file_lastUpdatedDate,object_id,object_name,object_type
0,line:IDFM:C01423,1,1,Bus,network:IDFM:6,2024-12-06T13:50:22.638Z,line:IDFM:C01423,1,line
1,line:IDFM:C01426,2,2,Bus,network:IDFM:6,2024-12-06T13:50:22.638Z,line:IDFM:C01426,2,line
2,line:IDFM:C02129,3,3,Bus,network:IDFM:6,2024-12-06T13:50:22.638Z,line:IDFM:C02129,3,line
3,line:IDFM:C02130,3s,3s,Bus,network:IDFM:6,2024-12-06T13:50:22.638Z,line:IDFM:C02130,3s,line
4,line:IDFM:C02137,4,4,Bus,network:IDFM:6,2024-12-06T13:50:22.638Z,line:IDFM:C02137,4,line


In [81]:
df_objects["object_type"].unique()

array(['line', 'stop_point', 'network', 'stop_area'], dtype=object)

In [83]:
print("Nombre de lignes :", len(df_objects[df_objects["object_type"] == "line"]))
print("Nombre d'arrêts :", len(df_objects[df_objects["object_type"] == "stop_point"]))
print("Nombre de réseaux :", len(df_objects[df_objects["object_type"] == "network"]))
print("Nombre de tronçons :", len(df_objects[df_objects["object_type"] == "stop_area"]))
print("Modes de transport :", df_objects["line_mode"].unique())

Nombre de lignes : 943
Nombre d'arrêts : 6597
Nombre de réseaux : 5
Nombre de tronçons : 25
Modes de transport : ['Bus' 'Tramway' 'Metro' 'RapidTransit' 'LocalTrain' 'Funicular']


In [ ]:
df_lines = df_objects.drop_duplicates(subset=["line_id"]) # plusieurs arrêts ont la même ligne ce qui fausserait le compte

print("Nombre de lignes de bus :", len(df_lines[df_lines["line_mode"] == "Bus"]))
print("Nombre de lignes de tram :", len(df_lines[df_lines["line_mode"] == "Tramway"]))
print("Nombre de lignes de métro :", len(df_lines[df_lines["line_mode"] == "Metro"]))
print("Nombre de lignes de RER :", len(df_lines[df_lines["line_mode"] == "RapidTransit"]))
print("Nombre de lignes de Transilien :", len(df_lines[df_lines["line_mode"] == "LocalTrain"]))
print("Nombre de funiculaires :", len(df_lines[df_lines["line_mode"] == "Funicular"]))

Nombre de lignes de bus : 919
Nombre de lignes de tram : 13
Nombre de lignes de métro : 16
Nombre de lignes de RER : 5
Nombre de lignes de Transilien : 9
Nombre de funiculaires : 1


Le funiculaire de Montmartre (seul funiculaire du réseau) et toutes les lignes de métro et de RER sont représentées dans notre jeu de données. La moitié des lignes de transilien ainsi qu'un tram manquent à l'appel. Enfin, un peu moins des deux tiers des environ 1500 lignes de bus d'Île de France sont recensés dans notre jeu de données.

In [84]:
df_objects[df_objects["object_type"] == "network"]

,line_id,line_name,line_shortName,line_mode,line_networkId,file_lastUpdatedDate,object_id,object_name,object_type
94,line:IDFM:C00772,1201,1201,Bus,network:IDFM:1082,2024-12-06T13:50:22.638Z,network:IDFM:1082,Cergy-Pontoise Confluence,network
104,line:IDFM:C00086,1 (future 4511),1,Bus,network:IDFM:1076,2024-12-06T13:50:22.638Z,network:IDFM:1076,Cœur d’Essonne,network
461,line:IDFM:C00798,Express 46,46,Bus,network:IDFM:1073,2024-12-06T13:50:22.638Z,network:IDFM:1073,Pays de Montereau,network
4714,line:IDFM:C00380,24-06 (future 4277),24-06,Bus,network:IDFM:1080,2024-12-11T15:43:19.231Z,network:IDFM:1080,Evry Centre Essonne,network
6273,line:IDFM:C00074,1,1,Bus,network:IDFM:1060,2024-12-17T08:16:24.049Z,network:IDFM:1060,Paris Saclay,network


In [86]:
df_objects[df_objects["object_type"] == "stop_area"].head()

,line_id,line_name,line_shortName,line_mode,line_networkId,file_lastUpdatedDate,object_id,object_name,object_type
198,line:IDFM:C01571,Express 91-10,EX91-10,Bus,network:IDFM:1079,2024-12-06T13:50:22.638Z,stop_area:IDFM:63279,Rond-Point de Villiers - Maison Foujita,stop_area
2186,line:IDFM:C00299,1510,1510,Bus,network:IDFM:1049,2024-12-06T13:50:22.638Z,stop_area:IDFM:419400,Alouettes,stop_area
2187,line:IDFM:C00299,1510,1510,Bus,network:IDFM:1049,2024-12-06T13:50:22.638Z,stop_area:IDFM:65558,Alliance Bussys,stop_area
2188,line:IDFM:C00299,1510,1510,Bus,network:IDFM:1049,2024-12-06T13:50:22.638Z,stop_area:IDFM:65564,Bouquinvilles,stop_area
2189,line:IDFM:C00299,1510,1510,Bus,network:IDFM:1049,2024-12-06T13:50:22.638Z,stop_area:IDFM:65599,Piscine CDFAS,stop_area


La documentation de l'API IDF Mobilités nous enseigne que `stop_area` correspond à un regroupement d’arrêts physiques portant le même nom.

Le réseau de transports en commun d'Île de France est plus vaste que cela. Notre analyse ne couvrant que 3 semaines du mois de décembre, il n'est pas étonnant que l'ensemble des objets du réseau n'ait pas été affecté par une perturbation sur cette période.

## Identification des lignes les plus problématiques

On s'attend à priori à ce que les lignes les plus fréquentées (métro, RER...) soient celles qui subissent le plus de perturbations. Si cet hypothèse est exacte, nous tenterons également d'identifier les lignes de bus les plus problématiques.